In [14]:

import itertools
import numpy as np
from sympy import *
import torch

k = 2
seq_len = 20
ind = range(0,seq_len,1)
comb = []
for comb_length in range(2,k+1,1):
    # compute all combination of ind:
    comb.extend(list(itertools.combinations(ind,comb_length)))

#print(comb)

# fix the seed torch:
torch.manual_seed(0)
# create a random torch tensor of shape (3, seq_len):
u_f = torch.randn(3, seq_len)

# create an empty torch tensor of length len(comb) with first dimention the same as u_f:
u_f_corr = torch.zeros(u_f.shape[0], len(comb))
for i in range (0,len(comb),1):
    u_f_corr[..., i] = u_f[..., comb[i][0]] * u_f[..., comb[i][1]]   


# flip u_f_corr along its last dimension:
u_f_corr_flip = torch.flip(u_f_corr, [-1])

# A, B, u0, u1, u2, u3, u4, u5, u6, u7, u8 = symbols('A B u0 u1 u2 u3 u4 u5 u6 u7 u8')

# expr0 = B * u0

# expr1 = B * u1  + (A + B * u1) * expr0
# xx = expand(expr1)

# expr2 = B * u2  + (A + B * u2) * xx
# expand(expr2)

# # expr3 = B * u3  + (A + B * u3) * xx
# # xx = expand(expr3)

# # expr4 = B * u4  + (A + B * u4) * xx
# # xx = expand(expr4)

# # expr5 = B * u5  + (A + B * u5) * xx
# # xx = expand(expr5)

# # expr6 = B * u6  + (A + B * u6) * xx
# # xx = expand(expr6)

# # expr7 = B * u7  + (A + B * u7) * xx
# # xx = expand(expr7)

# # expr8 = B * u8  + (A + B * u8) * xx
# # expand(expr8)




In [1]:
import sys
import pathlib

p = pathlib.Path().absolute()
print("Adding path: ", p)
sys.path.append(str(p))
sys.path.append(str('/home/ramin/state-spaces/'))

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.fft
from einops import rearrange, repeat
from opt_einsum import contract, contract_expression
from omegaconf import DictConfig

import src.models.hippo.hippo as hippo
from src.models.functional.krylov import krylov, power

import src.utils.train

log = src.utils.train.get_logger()

try:
    from extensions.cauchy.cauchy import cauchy_mult
    has_cauchy_extension = True
except:
    log.warn(
        "CUDA extension for cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%"
    )
    has_cauchy_extension = False

try:
    import pykeops
    from src.models.functional.cauchy import cauchy_conj
    has_pykeops = True
except ImportError:
    has_pykeops = False
    from src.models.functional.cauchy import cauchy_slow
    if not has_cauchy_extension:
        log.error(
            "Falling back on slow Cauchy kernel. Install at least one of pykeops or the CUDA extension for efficiency."
        )

_isnan = lambda x: torch.isnan(x).any()
_isinf = lambda x: torch.isinf(x).any()

# conj creates a vector tensor of x and its conjugate:
_conj = lambda x: torch.cat([x, x.conj()], dim=-1)

_c2r = torch.view_as_real
_r2c = torch.view_as_complex

# resolve_conj takes the conjucate of a given complex tensor:
if tuple(map(int, torch.__version__.split('.')[:2])) >= (1, 10):
    _resolve_conj = lambda x: x.conj().resolve_conj()
else:
    _resolve_conj = lambda x: x.conj()

def bilinear(dt, A, B=None):
    """
    dt: (...) timescales
    A: (... N N)
    B: (... N)
    """
    N = A.shape[-1]
    I = torch.eye(N).to(A)
    A_backwards = I - dt[:, None, None] / 2 * A
    A_forwards = I + dt[:, None, None] / 2 * A

    if B is None:
        dB = None
    else:
        dB = dt[..., None] * torch.linalg.solve(
            A_backwards, B.unsqueeze(-1)
        ).squeeze(-1) # (... N)

    dA = torch.linalg.solve(A_backwards, A_forwards)  # (... N N)
    return dA, dB

# fix the seed 
torch.manual_seed(0)
dt = 0.0001 * torch.randn(10)
# create a 100 x 100 torch tensor:
A = torch.randn(10, 10)
B = torch.randn(10)

dA, dB = bilinear(dt, A, B)

Adding path:  /home/ramin/state-spaces/src/models/sequence/ss


/home/ramin/miniconda3/envs/state-space/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_701559/1005520754.py:30: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn(
CUDA extension for cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%
